This notebook is only for reference, don't make changes in it. Copy it to your drive (e.g. in ybshmmlchk) and use there.

In [ ]:
######################
### INITIALIZATION ###
######################
# mount Google Drive and switch to zatoboj folder in Files view
from google.colab import drive
drive.mount('/content/gdrive')  
from google.colab import files
files.view('/content/gdrive/MyDrive/ybshmmlchk/zatoboj') 
# automatically mount PyDrive
import json
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
### FOR THE FIRST RUN: 
### You will authorize twice: once for mounting gdrive, once for pydrive.
### Then open 'adc.json' from 'content' folder and copy-paste client_id, client_secret and refresh_token into dict adc below and remove # signs on lines 19-21.
### After this is done you won't have to authorize for pydrive anymore.
# adc = {                                                                                                                                                      "client_id": "", "client_secret": "", "refresh_token": "1//093X5tMAwUbEGCgYIARAAGAkSNwF-L9IrKTrK23ol4d_X0SIwM2LytDMQfmwR5M9F22I33OXZ_3RthEsqHoXJCSfl5YGSwDKHuSg", "revoke_uri": "", "scopes": ["openid","https://www.googleapis.com/auth/userinfo.email","https://www.googleapis.com/auth/cloud-platform","https://www.googleapis.com/auth/appengine.admin","https://www.googleapis.com/auth/compute","https://www.googleapis.com/auth/accounts.reauth","https://www.googleapis.com/auth/drive"],"token_uri": "https://oauth2.googleapis.com/token","type": "authorized_user"}
# with open('adc.json', 'w') as f:
#     json.dump(adc, f)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
py_drive = GoogleDrive(gauth)
# if you want to work with git from this notebook
%cd /content/gdrive/My\ Drive/ybshmmlchk/zatoboj
!mkdir -p ~/.ssh
!cp ../id_rsa ~/.ssh
!ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts
!ssh -T git@github.com
!source ../git_config_startup.sh
# install module ztpt
!pip install -q -q -e .
# import module ztpt
import ztpt
# set up module autoreload
%load_ext autoreload
%autoreload 2
# check your GPU
!nvidia-smi

In [ ]:
!git pull
!git status

In [ ]:
!git add .
!git commit -m 'your comment'
!git push

[main 4f6365d] delete models from Trash with pydrive
 2 files changed, 9 insertions(+), 2 deletions(-)
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 632 bytes | 316.00 KiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To github.com:zatoboj/zatoboj.git
   55eed6e..4f6365d  main -> main


This is configuration of your run.

In [ ]:
'''
Model name is automatically set to:

{transformer.model}-
{transformer.version}-
{model.model}=
{model.unique_name}-
{model.max_len}-
{model.batch_size}-
{model.lr}-
{model.freeze_layers}.ckpt
'''

config = ztpt.default_config()

config.dirs.py_drive = py_drive

config.transformer.model = 'albert'                 # default: albert
config.transformer.version = 'large-v2'             # default: base-v2

config.model.max_len = 256                          # default: 256
config.model.batch_size = 10                        # default: 16
config.model.model = 'SQUADBERT'                    # default: 'SQUADBERT'
config.model.lr = 1e-05                             # default: 1e-05
config.model.freeze_layers = 0                      # default: 0
config.model.signature = 'vostok-2-3xbatch'         # default: '' (can be: {your_name}-{version})

config.train.max_epochs = 3                         # default: 1
config.train.val_check_interval = 0.2               # default: 0.2
config.train.accumulate_grad_batches = 3
# config.train.limit_train_batches = 0.2            # default: 1.
# config.train.limit_val_batches = 0.2              # default: 1.

In [ ]:
ztpt.train_model(config)

This is you gym, where you can use `ztpt` module to train and test models. Module `ztpt` includes:


*   `preprocess(config)` -- preprocesses and saves dataset from configuration, unless it already exists on drive
*   `load_data(config)` -- return train,val,test datasets. Load existing or runs preprocessing if needed
*   `create_model(config)` -- return model from configuration. If such model already exists, will suggest to change the name. If you want to load existing model use `load_model`
*   `load_model(config)` -- return saved model, ready to continue traning (todo: set weights&biases to continue the plots, not create new ones)
*   `train_model(config)` -- train model and save result. Everything that preceed traning (preprocessing, creating, loading etc) is done automatically.
*   `evaluate(config)` -- evalute model and show dictionary of accuracy results.